In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from crewai import Agent
from crewai_tools import SerperDevTool
from crewai import Task
from crewai import Crew, Process
from crewai import LLM


In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
# Access the environment variables
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_API_BASE = os.getenv("GROQ_API_BASE") 
GROQ_MODEL = os.getenv("GROQ_MODEL","groq/llama-3.3-70b-versatile")


sender_email = os.getenv("SENDER_EMAIL")
sender_password = os.getenv("SENDER_PASSWORD")
recipient_email = os.getenv("RECIPIENT_EMAIL")

In [4]:
search_tool = SerperDevTool()


In [5]:

import os
from crewai import LLM

groq_llm = LLM(
        model = GROQ_MODEL,
        api_key = GROQ_API_KEY,
        base_url = GROQ_API_BASE,
        temperature=0.3, 
        max_tokens=1024)

llm = groq_llm

In [6]:
# Define your critique logic
def critique_text(text):
    prompt = (
        "Critically review the following content. Your review must cover:\n"
        "- Grammar and spelling errors\n"
        "- Clarity and coherence of ideas\n"
        "- Tone appropriateness\n"
        "- Ethical soundness\n"
        "- Legal accuracy\n"
        "- Bias detection\n"
        "- Factual correctness\n\n"
        f"Text to critique:\n{text}\n\n"
        "Return a concise critique report with suggested improvements."
    )
    return llm(prompt) 

self_critique_tool = {
    "name": "SelfCritiqueTool",
    "description": "Critiques writing for grammar, clarity, tone, bias, and factual accuracy.",
    "function": critique_text
}





In [7]:
research_writer = Agent(
    role="Researcher & Writer",
    goal=(
        "Research cutting-edge developments in {topic}, write insightful reports, "
        "and ensure content quality by performing a thorough critique before delivery."
    ),
    verbose=True,
    memory=True,
    backstory="An expert who researches, writes, and ensures reports are accurate, clear, and ethically sound.",
    tools=[search_tool],
    allow_delegation=True,
    llm=llm
)


In [ ]:

research_writer_task = Task(
    description=(
   "Conduct web research on the topic: '{topic}'. Gather accurate and up-to-date information from credible sources. "
    "Then, convert the research findings into a professional, well-organized report suitable for PDF format. "
    "Structure the report using clear headings and subheadings for each subtopic. "
    "Use bullet points for clarity where needed, ensure smooth flow of ideas, and include properly formatted in-text citations and a references section."

    ),
    expected_output=(
       "- A polished, print-ready PDF report that includes:\n"
       "- A formal summary of findings\n"
       "- Sections organized by subtopics with clear headings\n"
       "- Bullet points or numbered lists for clarity\n"
       "- Inline hyperlinks to cited sources\n"
       "- A properly formatted references section at the end\n"
       "- Clear, professional, and accessible language throughout"
    ),
    tools=[search_tool], 
    agent=research_writer,
    output_file="final_research_report.pdf"
)


In [9]:
# Forming the crew with enhanced configurations
crew = Crew(
    agents=[research_writer],
    tasks=[research_writer_task],
    process=Process.sequential 
)

# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'The role of AI in improving food security in Africa'})
print(result)

# Agent: Researcher & Writer
## Task: Conduct web research on the topic: 'The role of AI in improving food security in Africa'. Gather accurate and up-to-date information from credible sources. Then, convert the research findings into a well-structured, readable report. Organize the report into clear sections by subtopics and ensure it is written in markdown format for easy readability.


# Agent: Researcher & Writer
## Thought: Thought: To conduct web research on the topic 'The role of AI in improving food security in Africa', I need to gather accurate and up-to-date information from credible sources. The first step is to search for relevant information on the internet.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"The role of AI in improving food security in Africa\"}"
## Tool Output: 
{'searchParameters': {'q': 'The role of AI in improving food security in Africa', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Is Arti

In [ ]:
import re
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from datetime import datetime
import os

def markdown_to_plaintext(line):
    # Convert markdown headers
    if line.startswith("## "):
        return f"<b>{line.replace('## ', '').strip().upper()}</b>"
    elif line.startswith("# "):
        return f"<b>{line.replace('# ', '').strip().upper()}</b>"
    # Convert bullets
    line = re.sub(r"^\* ", "• ", line)
    # Convert bold
    line = re.sub(r"\*\*(.*?)\*\*", r"<b>\1</b>", line)
    return line.strip()

def generate_pdf_report(content, filename="final_report.pdf", folder="reports",
                        author_name="Osita Wisdom Chinedu",
                        project_title="AI Research on Food Security in Africa"):

    if not os.path.exists(folder):
        os.makedirs(folder)

    filepath = os.path.join(folder, filename)
    doc = SimpleDocTemplate(filepath, pagesize=A4,
                            rightMargin=50, leftMargin=50,
                            topMargin=50, bottomMargin=50)

    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=4, leading=14, fontSize=11))
    styles.add(ParagraphStyle(name='CustomTitle', parent=styles['Heading1'], alignment=1, fontSize=16, spaceAfter=12))
    styles.add(ParagraphStyle(name='CustomSubtitle', alignment=1, fontSize=12, spaceAfter=6))

    flowables = []
    flowables.append(Paragraph(project_title, styles['CustomTitle']))
    flowables.append(Paragraph(f"Author: {author_name}", styles['CustomSubtitle']))
    flowables.append(Paragraph(f"Date: {datetime.now().strftime('%B %d, %Y')}", styles['CustomSubtitle']))
    flowables.append(Spacer(1, 12))

    for line in content.strip().split("\n"):
        if line.strip() == "":
            flowables.append(Spacer(1, 10))
        else:
            clean_line = markdown_to_plaintext(line)
            flowables.append(Paragraph(clean_line, styles['Justify']))

    doc.build(flowables)
    print(f"✅ PDF report generated at: {filepath}")
    return filepath


In [11]:
# Add Auto-Email Feature 
import yagmail
import os

def send_email_with_attachment(receiver_email, subject, body, attachment_path):

    sender_email = os.getenv("SENDER_EMAIL")
    sender_password = os.getenv("SENDER_PASSWORD")
   

    if not sender_email or not sender_password:
        raise ValueError("Please set SENDER_EMAIL and SENDER_PASSWORD in your environment variables.")

    yag = yagmail.SMTP(sender_email, sender_password)
    yag.send(to=receiver_email, subject=subject, contents=body, attachments=attachment_path)

    print(f"\n📧 Email sent to {receiver_email} with the PDF attachment.")

In [12]:
# GENERATE PDF
pdf_path = generate_pdf_report(
    content=str(result),
    filename="AI_Research_Report.pdf",
    author_name="Osita Wisdom Chinedu",
    project_title="AI Research on Food Security in Africa"
)

✅ PDF report generated at: reports\AI_Research_Report.pdf


In [13]:
# SEND EMAIL
send_email_with_attachment(
    receiver_email=recipient_email,
    subject="AI Research Report",
    body="Attached is your finalized research report.",
    attachment_path=pdf_path)


📧 Email sent to ositawisdomc@gmail.com with the PDF attachment.
